In [1]:
import pandas
from bs4 import BeautifulSoup

In [2]:
database="monsters.csv"
monsters = pandas.read_csv(database)
monsters = monsters.set_index('url')
monsters

,name,meta,str,dex,con,int,wis,cha,HP,AC,img,skills,senses,languages,descriptions
url,,,,,,,,,,,,,,,


In [8]:
monsters.index

Index([], dtype='object', name='url')

In [4]:
monsters_to_add = ['https://www.dndbeyond.com/monsters/16841-dire-wolf']

In [15]:
def add_monster(monsters, url):
    if url in monsters.index:
        print("found {url}, dropping")
        monsters = monsters.drop(url, axis='index')
    

In [16]:
add_monster(monsters, monsters_to_add[0])

In [17]:
from splinter import Browser
browser = Browser('firefox')
url = monsters_to_add[0]
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
browser.quit()

In [23]:
main_block = soup.find("div", class_="mon-stat-block")

In [24]:
print(main_block)

<div class="mon-stat-block">
<div class="mon-stat-block__header">
<div class="mon-stat-block__name">
<a class="mon-stat-block__name-link" href="/monsters/16841-dire-wolf">
                Dire Wolf
            </a>
</div>
<div class="mon-stat-block__meta">Large Beast, Unaligned</div>
</div>
<div class="mon-stat-block__separator">
<img alt="" class="mon-stat-block__separator-img" src="https://www.dndbeyond.com/file-attachments/0/579/stat-block-header-bar.svg"/>
</div>
<div class="mon-stat-block__attributes">
<div class="mon-stat-block__attribute">
<span class="mon-stat-block__attribute-label">Armor Class</span>
<span class="mon-stat-block__attribute-value">
<span class="mon-stat-block__attribute-data-value">
                    14
                </span>
<span class="mon-stat-block__attribute-data-extra">
                        (natural armor)  
                    </span>
</span>
</div>
<div class="mon-stat-block__attribute">
<span class="mon-stat-block__attribute-label">Hit Points</s

In [59]:
tidbits_str = ""
tidbits = main_block.find("div", class_="mon-stat-block__tidbits").find_all("div", class_="mon-stat-block__tidbit")
for tidbit in tidbits:
    label = tidbit.find("span", class_="mon-stat-block__tidbit-label").text.strip()
    data = tidbit.find("span", class_="mon-stat-block__tidbit-data").text.strip()
    tidbits_str += f";{label}: {data}"
tidbits_str = tidbits_str[1:]
tidbits_str

'Skills: Perception +3, Stealth +4;Senses: Passive Perception 13;Languages: --;Challenge: 1 (200 XP);Proficiency Bonus: +2'

In [53]:
def get_attribute(block):
    header = block.find("div", class_="ability-block__heading").text.strip()
    value = block.find("span", class_="ability-block__score").text.strip()
    mod = block.find("span", class_="ability-block__modifier")
    if mod != None:
        value += f" {mod.text.strip()}"
    return header,value

stat_blocks = main_block.find("div", class_="mon-stat-block__stat-block")
h,strength = get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--str"))
h,dex = get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--dex"))
h,con = get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--con"))
h,intelligence = get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--int"))
h,wis = get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--wis"))
h,cha = get_attribute(stat_blocks.find("div", class_="ability-block__stat ability-block__stat--cha"))


STR:17 (+3)


In [56]:
image_url = soup.find("img", class_="monster-image")["src"]
image_url

'https://www.dndbeyond.com/avatars/thumbnails/16/484/1000/1000/636376300478361995.jpeg'

In [26]:
name = main_block.find("div", class_="mon-stat-block__name").text.strip()
name

'Dire Wolf'

In [27]:
meta = main_block.find("div", class_="mon-stat-block__meta").text.strip()
meta

'Large Beast, Unaligned'

In [35]:
attributes_str = ""
attributes = main_block.find("div", class_="mon-stat-block__attributes").find_all("div", class_="mon-stat-block__attribute")
for attribute in attributes:
    label = attribute.find("span", class_="mon-stat-block__attribute-label").text.strip()
    value = attribute.find("span", class_="mon-stat-block__attribute-data-value").text.strip()
    dataextra = attribute.find("span", class_="mon-stat-block__attribute-data-extra")
    if not dataextra == None:
        value = f"{value} {dataextra.text.strip()}"
    attributes_str += f";{label}:{value}"
    print(f"{label}: {value}")
attributes_str = attributes_str[1:]
attributes_str

Armor Class: 14 (natural armor)
Hit Points: 37 (5d10 + 10)
Speed: 50 ft.


'Armor Class:14 (natural armor);Hit Points:37 (5d10 + 10);Speed:50 ft.'